<img alt="Colaboratory logo" height="45px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px">

<h1>Welcome to Colaboratory!</h1>

Colaboratory is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud.

With Colaboratory you can write and execute code, save and share your analyses, and access powerful computing resources, all for free from your browser.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
from scipy import interpolate
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import convolve
import math
from scipy.misc import imread, imshow
from scipy import ndimage

In [ ]:
data=pd.read_csv("gdrive/My Drive/Train.csv")

In [ ]:
labels=data['label']
labels=labels.values
data=pd.read_csv("gdrive/My Drive/interpolated_imgs.csv")
data=data.drop(data.columns[0],axis=1)

In [ ]:
def interpolate1(k1):# An array of 1024
    k1=k1.reshape(32,32)
    x = np.arange(0,k1.shape[1])
    y = np.arange(0,k1.shape[0])
    #mask invalid values
    k1 = np.ma.masked_invalid(k1)
    xx, yy = np.meshgrid(x, y)
    #get only the valid values
    x1 = xx[~k1.mask]
    y1 = yy[~k1.mask]
    newarr = k1[~k1.mask]
    GD1 = interpolate.griddata((x1, y1), newarr.ravel(),(xx, yy),method='cubic')
    col_mean=np.nanmean(GD1,axis=0)
    inds=np.where(np.isnan(GD1))
    GD1[inds]=np.take(col_mean,inds[1])
    GD1=GD1.reshape(-1)
    #GD1=GD1.reshape(32,32)
    return GD1 #return a 32 x 32 array

In [ ]:
data=np.array(data)
data

array([[138.12647125, 137.19551933, 211.        , ..., 155.        ,
        154.70730515, 158.        ],
       [158.73110846, 149.45694296, 161.        , ..., 160.        ,
        160.        , 159.980359  ],
       [167.08552236, 161.09821499, 160.88490526, ..., 126.7600091 ,
        130.52920139, 130.        ],
       ...,
       [247.        , 244.        , 246.        , ..., 156.        ,
        152.87744867, 145.        ],
       [248.61311285, 255.        , 255.        , ..., 207.35527475,
        205.10617053, 209.        ],
       [116.        , 117.        , 117.91473394, ..., 169.        ,
        168.8497974 , 170.        ]])

In [ ]:
Xtrain=data[0:48500]
Ytrain=labels[0:48500]

In [ ]:
data2=pd.read_csv("gdrive/My Drive/inter_im.csv")
data2=data2.drop(data2.columns[0],axis=1)

In [ ]:
data2=np.array(data2)
data2.shape

(9700, 1024)

In [ ]:
Xtest=data2

In [ ]:

Xtrain = Xtrain.reshape(48500,32,32,1)
Xtest=Xtest.reshape(9700,32,32,1)

(48500, 32, 32, 1) (9700, 32, 32, 1) (48500,) (8500, 10)


In [ ]:

from keras.utils import to_categorical
Ytrain = to_categorical(Ytrain)
#Ytest=to_categorical(Ytest)
Ytrain.shape

(48500, 10)

In [ ]:
mean = np.mean(Xtrain,axis=(0,1,2,3))
std = np.std(Xtrain,axis=(0,1,2,3))
Xtrain = (Xtrain-mean)/(std+1e-7)
Xtest = (Xtest-mean)/(std+1e-7)

In [ ]:
num_classes = 10
weight_decay = 1e-4

In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.callbacks import LearningRateScheduler

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=Xtrain.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2

In [ ]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(Xtrain)
 

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [ ]:
#training
batch_size = 64
 
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(Xtrain, Ytrain, batch_size=batch_size),\
                    steps_per_epoch=Xtrain.shape[0] // batch_size,epochs=125,\
                    verbose=1,callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/125
625/625 [==============================] - 31s 50ms/step - loss: 1.1571 - acc: 0.6715
Epoch 2/125
625/625 [==============================] - 29s 46ms/step - loss: 1.1516 - acc: 0.6751
Epoch 3/125
625/625 [==============================] - 28s 45ms/step - loss: 1.1461 - acc: 0.6778
Epoch 4/125
625/625 [==============================] - 28s 44ms/step - loss: 1.1387 - acc: 0.6794
Epoch 5/125
625/625 [==============================] - 28s 44ms/step - loss: 1.1377 - acc: 0.6800
Epoch 6/125
625/625 [==============================] - 28s 44ms/step - loss: 1.1339 - acc: 0.6811
Epoch 7/125
625/625 [==============================] - 28s 44ms/step - loss: 1.1324 - acc: 0.6844
Epoch 8/125
625/625 [==============================] - 28s 44ms/step - loss: 1.1228 - acc: 0.6877
Epoch 9/125
625/625 [==============================] - 27s 44ms/step - loss: 1.1244 - acc: 0.6860
Epoch 10/125
625/625 [==============================] - 27s 43ms/step - loss: 1.1179 - acc: 0.6904
Epoch 11/125
625/62

In [ ]:
scores = model.evaluate(Xtest, Ytest, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

8500/8500 [==============================] - 1s 170us/step

Test result: 73.929 loss: 1.023


In [ ]:
pd.DataFrame(actual).to_csv('final.csv')

## More Resources

Learn how to make the most of Python, Jupyter, Colaboratory, and related tools with these resources:

### Working with Notebooks in Colaboratory
- [Overview of Colaboratory](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb) 
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas](/notebooks/mlcc/intro_to_pandas.ipynb)
- [Tensorflow concepts](/notebooks/mlcc/tensorflow_programming_concepts.ipynb)
- [First steps with TensorFlow](/notebooks/mlcc/first_steps_with_tensor_flow.ipynb)
- [Intro to neural nets](/notebooks/mlcc/intro_to_neural_nets.ipynb)
- [Intro to sparse data and embeddings](/notebooks/mlcc/intro_to_sparse_data_and_embeddings.ipynb)

### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TensorFlow with TPUs](/notebooks/tpu.ipynb)

## Machine Learning Examples: Seedbank

To see end-to-end examples of the interactive machine learning analyses that Colaboratory makes possible, check out the [Seedbank](https://research.google.com/seedbank/) project.

A few featured examples:

- [Neural Style Transfer](https://research.google.com/seedbank/seed/neural_style_transfer_with_tfkeras): Use deep learning to transfer style between images.
- [EZ NSynth](https://research.google.com/seedbank/seed/ez_nsynth): Synthesize audio with WaveNet auto-encoders.
- [Fashion MNIST with Keras and TPUs](https://research.google.com/seedbank/seed/fashion_mnist_with_keras_and_tpus): Classify fashion-related images with deep learning.
- [DeepDream](https://research.google.com/seedbank/seed/deepdream): Produce DeepDream images from your own photos.
- [Convolutional VAE](https://research.google.com/seedbank/seed/convolutional_vae): Create a generative model of handwritten digits.